In [ ]:
import random
import time

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.decomposition import PCA



import sys
!{sys.executable} -m pip install pygame
# !python pygame-1.9.6/setup.py
import draw_digits as game

In [ ]:

def img2vector(filename):

    returnVect = np.zeros((1, 1024))
    fr = open(filename)

    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])

    return returnVect
def plotNumbers(trainingMat, hwLabels):
    
    plt.scatter(trainingMat[:, 0], trainingMat[:, 1],
            c=hwLabels, edgecolor='none', alpha=0.5)
    plt.set_cmap(plt.cm.get_cmap('nipy_spectral',10))

    plt.xlabel('component 1')
    plt.ylabel('component 2')
    plt.colorbar()
    plt.show()

def drawYourOwnDigits():
    game.play()
    myDigits = listdir('myDigits')
    m = len(myDigits)
    myDigitMat = np.zeros((m, 1024))
    classNumbers = []
    for i in range(m):
        fileNameStr = myDigits[i]

        classNumbers.append(int(fileNameStr.split('_')[0]))
        myDigitMat[i,:] = img2vector('myDigits/%s' % (fileNameStr))
    return myDigitMat, classNumbers

In [ ]:
# data is shuffled and thus the training and testing samples are different each time 

hwLabels = []
trainingFileList = listdir('trainingDigits')
random.shuffle(trainingFileList)
testFileList = trainingFileList[0:200]
trainingFileList = trainingFileList[201:len(trainingFileList)-1]

m = len(trainingFileList)

trainingMat = np.zeros((m, 1024))
TestMat = np.zeros((200, 1024))
classNumbers = []

for i in range(m):
    fileNameStr = trainingFileList[i]

    classNumber = int(fileNameStr.split('_')[0])
    hwLabels.append(classNumber)
    trainingMat[i,:] = img2vector('trainingDigits/%s' % (fileNameStr))

for i in range(200):

    fileNameStr = testFileList[i]

    classNumbers.append(int(fileNameStr.split('_')[0]))

    TestMat[i,:] = img2vector('trainingDigits/%s' % (fileNameStr))

print(trainingMat.shape)
print(TestMat.shape)

In [ ]:
# run this part to make your own test cases :)
# make sure retrain the data before run this part
# Press s or Esc to finish drawing


TestMat, classNumbers = drawYourOwnDigits()

In [ ]:
# Run this part to pca the data

NumOfDimensions = 10



pca_model = PCA(n_components=NumOfDimensions)
pca_model.fit(trainingMat)
trainingMat = pca_model.transform(trainingMat)
TestMat = pca_model.transform(TestMat)
plotNumbers(trainingMat, hwLabels)


print(trainingMat.shape)
print(TestMat.shape)

In [ ]:
n_neighbors = 30


neigh =KNN(n_neighbors, algorithm = 'auto')
neigh.fit(trainingMat, hwLabels)
mTest = len(TestMat)



errorCount = 0.0
start_time = time.time()
classifierResult = neigh.predict(TestMat)
for i in range(mTest):
    if(classifierResult[i] != classNumbers[i] or mTest < 15):
        print("classified: %d\ real result:%d" % (classifierResult[i], int(classNumbers[i])))
        if(classifierResult[i] != classNumbers[i]):
            errorCount += 1.0
end_time = time.time()
print("Time used to predict was %g seconds" % (end_time - start_time))
print("Total misclassified data : %d \n in rate of%f%%" % (errorCount, errorCount/mTest * 100))
